In [698]:
import numpy as np
import pandas as pd

In [727]:
data = pd.read_csv('rub_usd.csv', sep=',')
#pd.to_datetime(raw_data['Mycol'], format='%d%b%Y:%H:%M:%S.%f')
data['Date'] = pd.to_datetime(data['Date'] + ' ' + data['<TIME>'])
p = 50
data2 = data.drop([i for i in range(len(data)-p, len(data))]) 
N = len(data2['Date'])
tau = int((N + 1) / 2) 
xt = np.array(data2['RUB/USD']) # ряд для построения модели

In [728]:
X = np.array([xt[i:(i+tau)] for i in range(N-tau+1)])
n = len(X[0])

In [729]:
C = 1 / n * X.transpose().dot(X)


In [730]:
w, V = np.linalg.eig(C)
W = np.diag(w)

In [731]:
#V.transpose().dot(C).dot(Vtau) # =W
#np.abs(np.linalg.det(C) - np.prod(w)) < 1e-3

In [732]:
Y = V.transpose().dot(X)

In [733]:
r = int(tau * 2 / 3)
Vr = V[:r]
Temp = Y.transpose()[:r]
Yr = Temp.transpose()

In [734]:
def S(P, V, r):
    return np.array([np.sum(P[i] * V[i]) for i in range(r)])

def findQ(xt, tau, N):
    return np.array([xt[i] for i in range(N-tau+1, N)]) 

def findNext(Vtau, Vstar, Q):
    return Vtau.dot(Vstar.transpose().dot(Q)) / (1 - Vtau.dot(Vtau.transpose()))

In [735]:
Q = findQ(xt, tau, N)
Vstar = np.array([V[i][:r] for i in range(tau-1)])
Vtau = np.array([V[len(V)-1][i] for i in range(r)])

In [736]:
#Vtau 912x608    -> VtauT 608x912   *   Q 912 -> M 912 
print(len(Vtau), len(Vstar), len(Q))

456 683 683


In [737]:
# cycle
NewN = N
NewXt = [x for x in xt]
for i in range(p):
    Q = findQ(NewXt, tau, NewN)
    x_next = findNext(Vtau, Vstar, Q)
    NewXt.append(x_next)
    NewN += 1 
    

In [738]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=data['Date'],
                y=data['RUB/USD'],
                name="Data",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=data['Date'],
                y=np.array(NewXt),
                name="Predict",
                line_color='red',
                opacity=0.8))

fig.show()